In [2]:
!pip install polars sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 42.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.2/613.2 KB 43.6 MB/s eta 0:00:00


In [1]:
import polars as pl
from sqlalchemy import create_engine

# read sqlite from ./data/items.db

conn = create_engine(f"sqlite:///data/items.db?immutable=1")


df = pl.read_database("select * from items order by id", conn.connect())

df

id,json
i64,str
0,"""{""id"":0,""delet…"
1,"""{""by"":""pg"",""de…"
2,"""{""by"":""phyllis…"
3,"""{""by"":""phyllis…"
4,"""{""by"":""onebeer…"
5,"""{""by"":""perler""…"
6,"""{""by"":""perler""…"
7,"""{""by"":""phyllis…"
8,"""{""by"":""frobnic…"


In [2]:
# iterate over the dataset 5M rows at a time, since polars can't handle more than that much JSON at once.
# https://github.com/pola-rs/polars/pull/11245

structured = []
batch_size = 5_000_000

for start_index in range(0, len(df), batch_size):
    print(start_index)
    output = (
        df.slice(start_index, batch_size)
        .with_columns(pl.col("json").str.json_extract())[["json"]]
        .unnest("json")
    )
    structured.append(output)

structured = pl.concat(structured, how="diagonal_relaxed").with_columns(
    pl.from_epoch("time", time_unit="s")
)

structured

0


5000000
10000000
15000000
20000000
25000000
30000000
35000000


id,deleted,by,descendants,kids,score,time,title,type,url,parent,text,dead
i64,bool,str,i64,list[i64],i64,datetime[μs],str,str,str,i64,str,bool
0,true,null,null,null,null,null,null,null,null,null,null,null
1,null,"""pg""",15,"[15, 234509, … 82729]",57,2006-10-09 18:21:51,"""Y Combinator""","""story""","""http://ycombin…",null,null,null
2,null,"""phyllis""",0,null,16,2006-10-09 18:30:28,"""A Student's Gu…","""story""","""http://www.pau…",null,null,null
3,null,"""phyllis""",0,[531602],7,2006-10-09 18:40:33,"""Woz Interview:…","""story""","""http://www.fou…",null,null,null
4,null,"""onebeerdave""",0,null,5,2006-10-09 18:47:42,"""NYC Developer …","""story""","""http://avc.blo…",null,null,null
5,null,"""perler""",0,null,7,2006-10-09 18:51:04,"""Google, YouTub…","""story""","""http://www.tec…",null,null,null
6,null,"""perler""",0,null,4,2006-10-09 18:56:40,"""Business Intel…","""story""","""http://360tech…",null,null,null
7,null,"""phyllis""",0,null,5,2006-10-09 19:00:55,"""Sevin Rosen Un…","""story""","""http://feature…",null,null,null
8,null,"""frobnicate""",0,null,10,2006-10-09 19:17:39,"""LikeBetter fea…","""story""","""http://news.bb…",null,null,null


In [6]:
structured[["id", "parent"]].with_columns(
    pl.col("parent").fill_null(pl.col("id"))
).head(5).to_pandas().set_index("id", drop=True).to_dict()["parent"]

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4}

In [7]:
parent_map = (
    structured[["id", "parent"]]
    .with_columns(pl.col("parent").fill_null(pl.col("id")))
    .to_pandas()
    .set_index("id", drop=True)
    .to_dict()["parent"]
)

In [8]:
def find_top_level_parent(node_id: int) -> int:
    if parent_map[node_id] == node_id:
        return node_id
    else:
        top_level_parent = find_top_level_parent(parent_map[node_id])
        parent_map[node_id] = top_level_parent  # Path compression
        return top_level_parent


structured = structured.with_columns(
    pl.col("id").map_elements(find_top_level_parent).alias("top_level_parent")
)[
    [
        "id",
        "type",
        "by",
        "time",
        "title",
        "text",
        "url",
        "score",
        "parent",
        "top_level_parent",
        "descendants",
        "kids",
        "deleted",
        "dead",
    ]
]

structured

id,type,by,time,title,text,url,score,parent,top_level_parent,descendants,kids,deleted,dead
i64,str,str,datetime[μs],str,str,str,i64,i64,i64,i64,list[i64],bool,bool
0,null,null,null,null,null,null,null,null,0,null,null,true,null
1,"""story""","""pg""",2006-10-09 18:21:51,"""Y Combinator""",null,"""http://ycombin…",57,null,1,15,"[15, 234509, … 82729]",null,null
2,"""story""","""phyllis""",2006-10-09 18:30:28,"""A Student's Gu…",null,"""http://www.pau…",16,null,2,0,null,null,null
3,"""story""","""phyllis""",2006-10-09 18:40:33,"""Woz Interview:…",null,"""http://www.fou…",7,null,3,0,[531602],null,null
4,"""story""","""onebeerdave""",2006-10-09 18:47:42,"""NYC Developer …",null,"""http://avc.blo…",5,null,4,0,null,null,null
5,"""story""","""perler""",2006-10-09 18:51:04,"""Google, YouTub…",null,"""http://www.tec…",7,null,5,0,null,null,null
6,"""story""","""perler""",2006-10-09 18:56:40,"""Business Intel…",null,"""http://360tech…",4,null,6,0,null,null,null
7,"""story""","""phyllis""",2006-10-09 19:00:55,"""Sevin Rosen Un…",null,"""http://feature…",5,null,7,0,null,null,null
8,"""story""","""frobnicate""",2006-10-09 19:17:39,"""LikeBetter fea…",null,"""http://news.bb…",10,null,8,0,null,null,null


In [10]:
to_push = structured.to_pandas()

to_push

,id,type,by,time,title,text,url,score,parent,top_level_parent,descendants,kids,deleted,dead
0,0,None,None,NaT,None,None,None,NaN,NaN,0,NaN,None,True,None
1,1,story,pg,2006-10-09 18:21:51,Y Combinator,None,http://ycombinator.com,57.0,NaN,1,15.0,"[15, 234509, 487171, 82729]",None,None
2,2,story,phyllis,2006-10-09 18:30:28,A Student's Guide to Startups,None,http://www.paulgraham.com/mit.html,16.0,NaN,2,0.0,None,None,None
3,3,story,phyllis,2006-10-09 18:40:33,Woz Interview: the early days of Apple,None,http://www.foundersatwork.com/stevewozniak.html,7.0,NaN,3,0.0,[531602],None,None
4,4,story,onebeerdave,2006-10-09 18:47:42,NYC Developer Dilemma,None,http://avc.blogs.com/a_vc/2006/10/the_nyc_deve...,5.0,NaN,4,0.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38109495,38109495,comment,pbnjay,2023-11-02 05:57:47,None,I don&#x27;t understand that copyright line if...,None,NaN,38104681.0,38099085,NaN,None,None,None
38109496,38109496,comment,canadianfella,2023-11-02 05:57:50,None,[dead],None,NaN,38109359.0,38100284,NaN,None,None,None
38109497,38109497,story,transpute,2023-11-02 05:57:50,Setting up a MikroTik 10 Gbps Switch the first...,None,https://www.jeffgeerling.com/blog/2023/setting...,1.0,NaN,38109497,0.0,None,None,None
38109498,38109498,comment,Jensson,2023-11-02 05:58:21,None,"Animals used to the winter survives being wet,...",None,NaN,38109103.0,38106461,NaN,None,None,None


In [12]:
import datasets as ds
from dotenv import load_dotenv

load_dotenv()

ds = ds.Dataset.from_pandas(to_push)
ds.push_to_hub("OpenPipe/hacker-news")

Pushing dataset shards to the dataset hub:   0%|          | 0/34 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1121 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1121 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1121 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1121 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1121 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1121 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1121 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1121 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/34 [00:00<?, ?it/s]